# Refactored `prodstatus` basics

## Logger configuration, part 1

Jupyter does funky things with the logger configuration. To change logger formatting (to add the time, in this case), you need to reload the logger before fiddling with `basicConfig`.
You only need to do this in jupyter notebooks, not python source files.

This must be done *before* the import of `lsst.prodstatus` or any of its submodules.

In [1]:
add_timestamp_to_logging = False
if add_timestamp_to_logging:
    from imp import reload
    reload(logging)

## Import of python modules

In [2]:
from tempfile import TemporaryDirectory
from unittest import mock
from os import environ
from pathlib import Path
import pprint
import dataclasses
import logging

In [3]:
import jira
import netrc
import pandas as pd
import numpy as np
import yaml

In [4]:
from lsst.ctrl.bps import BpsConfig
from lsst.prodstatus.Workflow import Workflow
from lsst.prodstatus.Step import Step
from lsst.prodstatus.Campaign import Campaign
from lsst.prodstatus import LOG

## Logger configuration, part 2

In a jupyter notebook, the code in "Logger configuration, part 1" needs to be run first for this to have any effect. In other python environments (e.g. running from a command line), "part 1" is not necessary.

In [5]:
if add_timestamp_to_logging:
    logging.basicConfig(format="%(asctime)s %(name)s %(levelname)s: %(message)s")

LOG.setLevel(logging.DEBUG)

In [6]:
LOG.debug('Foo')

## Misc notes on tools, philosophy, etc.

1. I implemented all of these classes using dataclasses. See [here](https://docs.python.org/3/library/dataclasses.html) for refenece documentation, and [here](https://realpython.com/python-data-classes/) for a tutorial.
2. When a function or method requires a certain kind of data, make the arguments represent that kind of data. For example, rather than take a file name or yaml string when a method requires a BPS configuration, made the method require an actualy instance of `lsst.control.bps.BpsConfig`. This means that the code inside is less likely to need to be update if the implementation of `BpsConfig` in `lsst.control.bps` changes, because it will autamatically use the new implementations. Similarly, I used [`pathlib.Path`](https://docs.python.org/3/library/pathlib.html) instead of plain strings for directory and file names (but the benefits are less obvious here).
3. I consistently tried to use [`tempfile.TemporaryDirectory`](https://docs.python.org/3/library/tempfile.html#tempfile.TemporaryDirectory) for making temporary directories, because it does a good job of automatically cleaning up after itself, so I never have to worry about where the temporary directory is, or explicitly delete it.
4. Store all tables and table-like data in `pandas.DataFrames`. This makes it easy to read and write from just about any machine readabale format one might want, plus easy to export string representations in just about any markup you want. In particular, you can use [`tabulate`](https://pypi.org/project/tabulate/) to export it into `jira` markup (or html, latex, or many others).
5. Never use accessors ("getters" and "setters"). In python, the problem solved by getters and setters in java is better solved using ["property" methods](https://docs.python.org/3/howto/descriptor.html#properties), and getters and setters just make the code more complicated. While in java it's often bad practice to use members directly, it's just fine in python. If you ever need to change the implementation of the class, just make the member a property method. ["Python is not Java"](https://dirtsimple.org/2004/12/python-is-not-java.html) has a good discussion of this and other things that are good practices in java (and C++) but bad practices in python.
6. Separate ("decouple") IO from code that does anything other than IO, so you can replace the IO methods without touching other stuff, and vice versa. That is, except for logging, never put IO in a method that does anything other than IO. (See [this talk](https://www.youtube.com/watch?v=DJtef410XaM).) This both makes the code easier to test, and will simplify switching to using a different storage method (e.g. a database) if we want or need to.
7. I take advantage of the [`__str__` "magic" python method](https://docs.python.org/3/reference/datamodel.html#object.__str__) to make sure all classes have a human readable output when printed. (Note that this is different from the [`__repr__` magic python method](https://docs.python.org/3/reference/datamodel.html#object.__repr__), which is expected to be a string representation that is valid python code to recreate the object in its current state.)

## Overview

The refactored architecture is build around three classes, `Campaign`, `Step`, and `Workflow`:

![prodstatus_class_diagram](prodstatus_classes1.png)

Instances of each of these classes can be saved to and loaded from either directories on local disk or jira issues.

## Workflows

### A minimal workflow

To create a workflow, I need to set a workflow name and get an instance of `BpsConfig`:

In [7]:
test_workflow_name = 'Alice'
bps_config_path = Path(
    environ["PRODSTATUS_DIR"], "tests", "data", "bps_config_base.yaml"
)
bps_config = BpsConfig(bps_config_path)

These are enought to make the required elements of a `Workflow` instance:

In [8]:
workflow = Workflow(bps_config, test_workflow_name)
print(workflow)

Workflow
name: Alice
issue name: None
step: None
band: all
BPS config dataQuery: instrument='LSSTCam-imSim' and skymap='DC2'
exposures: None



You can save it as files in a directory. For this notebook, I'll make a temporary directory (which will automatically get cleaned up when the notebook kernel is ended or restarted), but it "real life" you'd set `test_dir` to wherever you want to save the data:

In [9]:
test_dir_itself = TemporaryDirectory()
test_dir = test_dir_itself.name
test_dir

'/tmp/tmpzicispad'

In [10]:
workflow.to_files(test_dir)

Look at what files were created, and what was in them:

In [11]:
!find {test_dir}

/tmp/tmpzicispad
/tmp/tmpzicispad/Alice
/tmp/tmpzicispad/Alice/workflow.yaml
/tmp/tmpzicispad/Alice/bps_config.yaml


In [12]:
!cat {test_dir}/{test_workflow_name}/workflow.yaml

band: all
name: Alice


### Read it back again

In [13]:
reread_workflow = Workflow.from_files(test_dir, 'Alice')
print(reread_workflow)

Workflow
name: Alice
issue name: None
step: None
band: all
BPS config dataQuery: instrument='LSSTCam-imSim' and skymap='DC2'
exposures: None



### Split by band

I put the code to divide a workflow up into groups with in the `Workflow` class: starting with an instance of `Workflow` that does everything, a `split` method returns a list of instances of `Workflow` that do different subsets.

For example, to get a lists of `Workflow`s that together do everything that `workflow` does, but one band at a time:

In [14]:
band_workflows = workflow.split_by_band()

In [15]:
for workflow in band_workflows:
    print(workflow)

Workflow
name: Alice_u
issue name: None
step: None
band: u
BPS config dataQuery: (instrument='LSSTCam-imSim' and skymap='DC2') and (band == 'u')
exposures: None

Workflow
name: Alice_g
issue name: None
step: None
band: g
BPS config dataQuery: (instrument='LSSTCam-imSim' and skymap='DC2') and (band == 'g')
exposures: None

Workflow
name: Alice_r
issue name: None
step: None
band: r
BPS config dataQuery: (instrument='LSSTCam-imSim' and skymap='DC2') and (band == 'r')
exposures: None

Workflow
name: Alice_i
issue name: None
step: None
band: i
BPS config dataQuery: (instrument='LSSTCam-imSim' and skymap='DC2') and (band == 'i')
exposures: None

Workflow
name: Alice_z
issue name: None
step: None
band: z
BPS config dataQuery: (instrument='LSSTCam-imSim' and skymap='DC2') and (band == 'z')
exposures: None

Workflow
name: Alice_y
issue name: None
step: None
band: y
BPS config dataQuery: (instrument='LSSTCam-imSim' and skymap='DC2') and (band == 'y')
exposures: None



### Splitting into groups of exposures

To split an instance of `Workflow` by exposure id, the instance needs a table of exposures. So, set's make a table of exposures, and then create a new `Workflow` with this list:

In [16]:
def make_random_exposure_table(num_exposures, bands=tuple('ugrizy'), random_number_seed=42):
    random_number_generator = np.random.default_rng(random_number_seed)
    bands = random_number_generator.choice(np.array(bands), num_exposures)
    exp_ids = random_number_generator.choice(10000, num_exposures, replace=False)
    exposures = pd.DataFrame({'band': bands, 'exp_id': exp_ids}).sort_values('exp_id').set_index('exp_id', drop=False)
    return exposures

In [17]:
exposures = make_random_exposure_table(10, bands=('g', 'r'))
exposures

,band,exp_id
exp_id,,
1280,g,1280
4503,r,4503
5130,g,5130
5260,g,5260
7171,r,7171
7352,r,7352
7606,g,7606
7857,g,7857
8396,g,8396


Now, create a "big" instance of `Workflow` that does all the exposures:

In [18]:
big_workflow = Workflow(bps_config, 'Bob', exposures=exposures)

Now lets split this workflow up into several workflows, one for each band:

In [19]:
exp_workflows = big_workflow.split_by_exposure(4)
for workflow in exp_workflows:
    print(workflow)

Workflow
name: Bob_1
issue name: None
step: None
band: all
BPS config dataQuery: (instrument='LSSTCam-imSim' and skymap='DC2') and (exposure >= 1280) and (exposure <= 5260)
number of exposures: 4
min exposure id: 1280
max exposure id: 5260
exposure counts by band: {'g': 3, 'r': 1}

Workflow
name: Bob_2
issue name: None
step: None
band: all
BPS config dataQuery: (instrument='LSSTCam-imSim' and skymap='DC2') and (exposure >= 7171) and (exposure <= 7606)
number of exposures: 3
min exposure id: 7171
max exposure id: 7606
exposure counts by band: {'r': 2, 'g': 1}

Workflow
name: Bob_3
issue name: None
step: None
band: all
BPS config dataQuery: (instrument='LSSTCam-imSim' and skymap='DC2') and (exposure >= 7857) and (exposure <= 9748)
number of exposures: 3
min exposure id: 7857
max exposure id: 9748
exposure counts by band: {'g': 2, 'r': 1}



## Step

It is possible to create an instance of `Step` by making a list of instances of `Workflow` yourself, and using the default constructor:

In [20]:
workflows = [
    Workflow(bps_config, 'Alice'),
    Workflow(bps_config, 'Bob')]
step = Step('Carol', workflows=workflows)
print(step)

Step
name: Carol
issue name: None
split bands: False
exposure groups: None
workflows:
 - Alice (issue None) with dataQuery instrument='LSSTCam-imSim' and skymap='DC2'
 - Bob (issue None) with dataQuery instrument='LSSTCam-imSim' and skymap='DC2'


Normally, though, one would use `Step.generate_new` to take parameters for building a set of workflows, and create an instance of `Step` with the correct workflows (and also saving the parameters used to make the them).

First, set parameters to be used for dividing the workflow into subsets by exposure (parameters passed to `Workflow.split_by_exposure`):

In [21]:
exposure_groups = {
    'group_size': 4,
    'skip_groups': 0,
    'num_groups': None
}

Note that `skip_groups` defaults to 0, so it could be left off. A `num_groups` of `None` indicates "as many as it takes", and the default is `None`, so this could be left off as well:

In [22]:
help(Workflow.split_by_exposure)

Help on function split_by_exposure in module lsst.prodstatus.Workflow:

split_by_exposure(self, group_size=None, skip_groups=0, num_groups=None)
    Split the workflow by exposure number.
    
    Parameters
    ----------
    group_size : `int` optional
        The approximate size of the group. The default is None, which
        causes the method to return a single workflow with all
        exposures.
    skip_groups : `int` optional
        The number of groups to skip. The default is 0 (no skipped groups).
    num_groups : `int` optional
        The maximum number for groups. The default is None,
        for all groups
    
    Returns
    -------
    workflows : `List[Workflow]`
        A list of workflows.



Now, let's make a `Step`:

In [23]:
step = Step.generate_new('Dave', base_bps_config=bps_config, split_bands=True, exposure_groups=exposure_groups, exposures=exposures, workflow_base_name='wf')

In [24]:
print(step)

Step
name: Dave
issue name: None
split bands: True
exposure groups: {'group_size': 4, 'skip_groups': 0, 'num_groups': None}
workflows:
 - wf_Dave_g_1 (issue None) with dataQuery ((instrument='LSSTCam-imSim' and skymap='DC2') and (band == 'g')) and (exposure >= 1280) and (exposure <= 5260)
 - wf_Dave_g_2 (issue None) with dataQuery ((instrument='LSSTCam-imSim' and skymap='DC2') and (band == 'g')) and (exposure >= 7606) and (exposure <= 8396)
 - wf_Dave_r (issue None) with dataQuery (instrument='LSSTCam-imSim' and skymap='DC2') and (band == 'r')


We can save and reload the step much like we did with the `Workflow` example above. Note that saving an instance of `Step` saves all its workflows as well.

Again, use a temporary directory:

In [25]:
test_dir_itself = TemporaryDirectory()
test_dir = test_dir_itself.name
test_dir

'/tmp/tmpw7ixelzj'

In [26]:
step.to_files(test_dir)

In [27]:
!find {test_dir}

/tmp/tmpw7ixelzj
/tmp/tmpw7ixelzj/Dave
/tmp/tmpw7ixelzj/Dave/step.yaml
/tmp/tmpw7ixelzj/Dave/workflows
/tmp/tmpw7ixelzj/Dave/workflows/wf_Dave_r
/tmp/tmpw7ixelzj/Dave/workflows/wf_Dave_r/workflow.yaml
/tmp/tmpw7ixelzj/Dave/workflows/wf_Dave_r/bps_config.yaml
/tmp/tmpw7ixelzj/Dave/workflows/wf_Dave_r/explist.txt
/tmp/tmpw7ixelzj/Dave/workflows/wf_Dave_g_1
/tmp/tmpw7ixelzj/Dave/workflows/wf_Dave_g_1/workflow.yaml
/tmp/tmpw7ixelzj/Dave/workflows/wf_Dave_g_1/bps_config.yaml
/tmp/tmpw7ixelzj/Dave/workflows/wf_Dave_g_1/explist.txt
/tmp/tmpw7ixelzj/Dave/workflows/wf_Dave_g_2
/tmp/tmpw7ixelzj/Dave/workflows/wf_Dave_g_2/workflow.yaml
/tmp/tmpw7ixelzj/Dave/workflows/wf_Dave_g_2/bps_config.yaml
/tmp/tmpw7ixelzj/Dave/workflows/wf_Dave_g_2/explist.txt


In [28]:
!cat {test_dir}/{step.name}/step.yaml

exposure_groups:
    group_size: 4
    num_groups: null
    skip_groups: 0
name: Dave
split_bands: true
workflows:
-   issue: null
    name: wf_Dave_g_1
-   issue: null
    name: wf_Dave_g_2
-   issue: null
    name: wf_Dave_r


## Creating a campaign

Instances of `Campaign` can be created in a way analogous to `Workflow` and `Step`, by creating a list of instances of `Step` and calling the `Campaign` constructor:

In [29]:
eve_workflows = [
    Workflow(bps_config, 'Graham'),
    Workflow(bps_config, 'John')
]
frank_workflows = [
    Workflow(bps_config, 'Terry'),
    Workflow(bps_config, 'Michael')
]
grace_steps = [
    Step('Eve', workflows=eve_workflows),
    Step('Frank', workflows=frank_workflows)
]
campaign = Campaign('Grace', steps=grace_steps)
print(campaign)

Campaign
name: Grace
issue name: None
steps:
 - Eve (issue None) with 2 workflows
 - Frank (issue None) with 2 workflows


The `Campaign.create_from_yaml` method creates a new instance of `Campaign` by reading everything from files.

To show how this works, start by creating a set of files that can be loaded, beginning by creating a temprorary directory in which to keep them:

In [30]:
campaign_def_dir = TemporaryDirectory()
campaign_def_dir_path = Path(campaign_def_dir.name)
campaign_def_dir_path

PosixPath('/tmp/tmpb70uk5pc')

Now, create a PBS config file in this directory:

In [31]:
bps_config_path = campaign_def_dir_path.joinpath('base_bps_config.yaml')
with bps_config_path.open('wt') as bps_config_io:
    bps_config.dump(bps_config_io)
bps_config_path

PosixPath('/tmp/tmpb70uk5pc/base_bps_config.yaml')

In [32]:
!head {bps_config_path}

campaign: v23_0_0_rc5/PREOPS-938
computeSite: LSST
defaultPreCmdOpts: --long-log --log-level=VERBOSE --log-file payload-log.json
executionButler:
  queue: DOMA_LSST_GOOGLE_MERGE
iddsServer: https://aipanda015.cern.ch:443/idds
payload:
  butlerConfig: s3://butler-us-central1-panda-dev/dc2/butler-external.yaml
  dataQuery: instrument='LSSTCam-imSim' and skymap='DC2'
  fileDistributionEndPoint: s3://butler-us-central1-panda-dev/dc2/{payloadFolder}/{uniqProcName}/


We'll also need a file with a list of exposures there:

In [33]:
explist_path = campaign_def_dir_path.joinpath('explist.txt')
exposures.to_csv(explist_path, header=False, index=False, sep=" ")
explist_path

PosixPath('/tmp/tmpb70uk5pc/explist.txt')

In [34]:
!head {explist_path}

g 1280
r 4503
g 5130
g 5260
r 7171
r 7352
g 7606
g 7857
g 8396
r 9748


Finally, create the top level yaml file that describes the campaign:

In [35]:
campaign_config = {
    'exposures': explist_path.as_posix(),
    'name': 'test_ehn_2021-03-01',
    'steps': {
        'step1': {'base_bps_config': bps_config_path.as_posix(), 'exposure_groups': {}, 'split_bands': True},
        'step2': {'base_bps_config': bps_config_path.as_posix(), 'exposure_groups': {'group_size': 4}, 'split_bands': True}
    }
}

In [36]:
campaign_def_path = campaign_def_dir_path.joinpath('campaign.yaml')
with campaign_def_path.open('wt') as campaign_def_io:
    yaml.dump(campaign_config, campaign_def_io, indent=4)
campaign_def_path

PosixPath('/tmp/tmpb70uk5pc/campaign.yaml')

In [37]:
!cat {campaign_def_path}

exposures: /tmp/tmpb70uk5pc/explist.txt
name: test_ehn_2021-03-01
steps:
    step1:
        base_bps_config: /tmp/tmpb70uk5pc/base_bps_config.yaml
        exposure_groups: {}
        split_bands: true
    step2:
        base_bps_config: /tmp/tmpb70uk5pc/base_bps_config.yaml
        exposure_groups:
            group_size: 4
        split_bands: true


Look at all the file's we've made in preparation:

In [38]:
!find {campaign_def_dir_path}

/tmp/tmpb70uk5pc
/tmp/tmpb70uk5pc/campaign.yaml
/tmp/tmpb70uk5pc/base_bps_config.yaml
/tmp/tmpb70uk5pc/explist.txt


Now we can use these files to create the instance of `Campaign`, including all corresponding instances of `Step` and `Workflow`:

In [39]:
campaign = Campaign.create_from_yaml(campaign_def_path)

In [40]:
print(campaign)

Campaign
name: test_ehn_2021-03-01
issue name: None
steps:
 - step1 (issue None) with 2 workflows
 - step2 (issue None) with 3 workflows


In [41]:
for step in campaign.steps:
    print(step)
    print()
    for workflow in step.workflows:
        print(workflow)

Step
name: step1
issue name: None
split bands: True
exposure groups: {}
workflows:
 - test_ehn_2021-03-01_step1_g (issue None) with dataQuery (instrument='LSSTCam-imSim' and skymap='DC2') and (band == 'g')
 - test_ehn_2021-03-01_step1_r (issue None) with dataQuery (instrument='LSSTCam-imSim' and skymap='DC2') and (band == 'r')

Workflow
name: test_ehn_2021-03-01_step1_g
issue name: None
step: step1
band: g
BPS config dataQuery: (instrument='LSSTCam-imSim' and skymap='DC2') and (band == 'g')
number of exposures: 6
min exposure id: 1280
max exposure id: 8396
exposure counts by band: {'g': 6}

Workflow
name: test_ehn_2021-03-01_step1_r
issue name: None
step: step1
band: r
BPS config dataQuery: (instrument='LSSTCam-imSim' and skymap='DC2') and (band == 'r')
number of exposures: 4
min exposure id: 4503
max exposure id: 9748
exposure counts by band: {'r': 4}

Step
name: step2
issue name: None
split bands: True
exposure groups: {'group_size': 4}
workflows:
 - test_ehn_2021-03-01_step2_g_1 (is

As with instances of `Step` and `Workflow`, we can save the instance of `Campaign` (including all the instances of `Step` and `Workflow` it contains) to files in a directory.

Here, I create a temporary directory to save them to (and it will be automatically cleaned up), but in "real life" you'd use the actual directory where you want it:

In [42]:
campaign_dir = TemporaryDirectory()
campaign_dir_path = Path(campaign_dir.name)
campaign_dir_path

PosixPath('/tmp/tmp4qzmhpu1')

In [43]:
campaign.to_files(campaign_dir_path)

In [44]:
!find {campaign_dir_path}

/tmp/tmp4qzmhpu1
/tmp/tmp4qzmhpu1/test_ehn_2021-03-01
/tmp/tmp4qzmhpu1/test_ehn_2021-03-01/campaign.yaml
/tmp/tmp4qzmhpu1/test_ehn_2021-03-01/steps
/tmp/tmp4qzmhpu1/test_ehn_2021-03-01/steps/step2
/tmp/tmp4qzmhpu1/test_ehn_2021-03-01/steps/step2/step.yaml
/tmp/tmp4qzmhpu1/test_ehn_2021-03-01/steps/step2/workflows
/tmp/tmp4qzmhpu1/test_ehn_2021-03-01/steps/step2/workflows/test_ehn_2021-03-01_step2_r
/tmp/tmp4qzmhpu1/test_ehn_2021-03-01/steps/step2/workflows/test_ehn_2021-03-01_step2_r/workflow.yaml
/tmp/tmp4qzmhpu1/test_ehn_2021-03-01/steps/step2/workflows/test_ehn_2021-03-01_step2_r/bps_config.yaml
/tmp/tmp4qzmhpu1/test_ehn_2021-03-01/steps/step2/workflows/test_ehn_2021-03-01_step2_r/explist.txt
/tmp/tmp4qzmhpu1/test_ehn_2021-03-01/steps/step2/workflows/test_ehn_2021-03-01_step2_g_2
/tmp/tmp4qzmhpu1/test_ehn_2021-03-01/steps/step2/workflows/test_ehn_2021-03-01_step2_g_2/workflow.yaml
/tmp/tmp4qzmhpu1/test_ehn_2021-03-01/steps/step2/workflows/test_ehn_2021-03-01_step2_g_2/bps_config.yam

In [45]:
!cat {campaign_dir_path}/{campaign.name}/campaign.yaml

name: test_ehn_2021-03-01
steps:
    step1:
        exposure_groups: {}
        issue: null
        split_bands: true
    step2:
        exposure_groups:
            group_size: 4
        issue: null
        split_bands: true


## Modification of campaigns, steps, and workflows

The `steps` member of a `Campaign` and the `workflows` member of `Step` are just lists of the corresponding objects: the user can add or remove them as desired.

Let's start by creating a new instance of `Campaign` as a starting point (repeating what we did above):

In [46]:
eve_workflows = [
    Workflow(bps_config, 'Graham'),
    Workflow(bps_config, 'John')
]
frank_workflows = [
    Workflow(bps_config, 'Terry'),
    Workflow(bps_config, 'Michael')
]
grace_steps = [
    Step('Eve', workflows=eve_workflows),
    Step('Frank', workflows=frank_workflows)
]
campaign = Campaign('Grace', steps=grace_steps)
print(campaign)

Campaign
name: Grace
issue name: None
steps:
 - Eve (issue None) with 2 workflows
 - Frank (issue None) with 2 workflows


We can add a workflow to one of the steps in the `Campaign` defined above:

In [47]:
extra_workflow = Workflow(bps_config, 'Harry')
campaign.steps[1].workflows.append(extra_workflow)
print(campaign)

Campaign
name: Grace
issue name: None
steps:
 - Eve (issue None) with 2 workflows
 - Frank (issue None) with 3 workflows


Notice that the step named `Frank`, the second step in the list (so the one with index `1`) had 2 workflows in it before, and now it has 3, because we just added one.

In [48]:
print(campaign.steps[1])

Step
name: Frank
issue name: None
split bands: False
exposure groups: None
workflows:
 - Terry (issue None) with dataQuery instrument='LSSTCam-imSim' and skymap='DC2'
 - Michael (issue None) with dataQuery instrument='LSSTCam-imSim' and skymap='DC2'
 - Harry (issue None) with dataQuery instrument='LSSTCam-imSim' and skymap='DC2'


Similarly, we can build a whole new step and add it:

In [49]:
step = Step.generate_new('Ingrid', base_bps_config=bps_config, split_bands=True, exposure_groups=exposure_groups, exposures=exposures, workflow_base_name='wf')
campaign.steps.append(step)
print(campaign)

Campaign
name: Grace
issue name: None
steps:
 - Eve (issue None) with 2 workflows
 - Frank (issue None) with 3 workflows
 - Ingrid (issue None) with 3 workflows


We can also get rid of steps and workflows using `remove`, `del`, and other python commands that remove elements from a list:

In [50]:
del campaign.steps[1]
print(campaign)

Campaign
name: Grace
issue name: None
steps:
 - Eve (issue None) with 2 workflows
 - Ingrid (issue None) with 3 workflows


# Test saving a campaign to jira

Begin by cutting down our test campaign to just one step with just one workflow.

In [51]:
workflow = Workflow(bps_config, 'Kelly', exposures=exposures)
step = Step('Larry', workflows=[workflow], exposures=exposures)
campaign = Campaign('Mary', steps=[step])

Let's assign them issue name so we don't make extra issues in jira every time we run this notebook. (I could have done this when I created them, above, but I separate it out to make it easier to comment out if we want the notebook to create all new issues.)

In [52]:
workflow.issue_name = 'DRP-187'
step.issue_name = 'DRP-186'
campaign.issue_name = 'DRP-185'

Take a look at what we have created:

In [53]:
print(campaign)
print()
for this_step in campaign.steps:
    print(this_step)
    print()
    for this_workflow in this_step.workflows:
        print(this_workflow)

Campaign
name: Mary
issue name: DRP-185
steps:
 - Larry (issue DRP-186) with 1 workflows

Step
name: Larry
issue name: DRP-186
split bands: False
exposure groups: None
workflows:
 - Kelly (issue DRP-187) with dataQuery instrument='LSSTCam-imSim' and skymap='DC2'

Workflow
name: Kelly
issue name: DRP-187
step: None
band: all
BPS config dataQuery: instrument='LSSTCam-imSim' and skymap='DC2'
number of exposures: 10
min exposure id: 1280
max exposure id: 9748
exposure counts by band: {'g': 6, 'r': 4}



Now, connect to the jira client and actually save the campaign:

In [54]:
secrets = netrc.netrc()
username, account, password = secrets.authenticators("lsstjira")
jira_client = jira.JIRA(options={"server": account}, basic_auth=(username, password))

In [55]:
campaign_issue = campaign.to_jira(jira_client, replace=True, cascade=True)

Reread the campaign from jira, and see if we get back what we just saved:

In [56]:
reread_campaign = Campaign.from_jira(jira_client.issue('DRP-185'), jira_client) 

In [57]:
print(reread_campaign)
print()
for this_step in reread_campaign.steps:
    print(this_step)
    print()
    for this_workflow in this_step.workflows:
        print(this_workflow)

Campaign
name: Mary
issue name: DRP-185
steps:
 - Larry (issue DRP-186) with 1 workflows

Step
name: Larry
issue name: DRP-186
split bands: False
exposure groups: {}
workflows:
 - Kelly (issue DRP-187) with dataQuery instrument='LSSTCam-imSim' and skymap='DC2'

Workflow
name: Kelly
issue name: DRP-187
step: None
band: all
BPS config dataQuery: instrument='LSSTCam-imSim' and skymap='DC2'
number of exposures: 10
min exposure id: 1280
max exposure id: 9748
exposure counts by band: {'g': 6, 'r': 4}



Save our reread campaign to a directory, and take a look at what's in the files there:

In [58]:
reread_campaign_dir = TemporaryDirectory()
reread_campaign_dir_path = Path(reread_campaign_dir.name)
reread_campaign_dir_path

PosixPath('/tmp/tmphf9jxbkg')

In [59]:
reread_campaign.to_files(reread_campaign_dir_path)

In [60]:
 !find {reread_campaign_dir_path} -type f

/tmp/tmphf9jxbkg/Mary/campaign.yaml
/tmp/tmphf9jxbkg/Mary/steps/Larry/step.yaml
/tmp/tmphf9jxbkg/Mary/steps/Larry/workflows/Kelly/workflow.yaml
/tmp/tmphf9jxbkg/Mary/steps/Larry/workflows/Kelly/bps_config.yaml
/tmp/tmphf9jxbkg/Mary/steps/Larry/workflows/Kelly/explist.txt
/tmp/tmphf9jxbkg/Mary/steps/Larry/explist.txt


In [61]:
!cat {reread_campaign_dir_path}/{campaign.name}/campaign.yaml

issue: DRP-185
name: Mary
steps:
    Larry:
        exposure_groups: {}
        issue: DRP-186
        split_bands: false


In [62]:
!cat {reread_campaign_dir_path}/{campaign.name}/steps/{step.name}/step.yaml

exposure_groups: {}
issue: DRP-186
name: Larry
split_bands: false
workflows:
-   issue: DRP-187
    name: Kelly


In [63]:
!cat {reread_campaign_dir_path}/{campaign.name}/steps/{step.name}/explist.txt

g 1280
r 4503
g 5130
g 5260
r 7171
r 7352
g 7606
g 7857
g 8396
r 9748


In [64]:
!cat {reread_campaign_dir_path}/{campaign.name}/steps/{step.name}/workflows/{workflow.name}/workflow.yaml

band: all
issue_name: DRP-187
name: Kelly


## Ideas for future development

What I had in mind for future development:

1. Expand the `to_jira` methods to capture the relationships between `Campaign`s, `Step`s, and `Workflow`s by adding issue links between issues. (Seach for `issue_link` in the [python API](https://jira.readthedocs.io/api.html).)
2. Add new methods to `Campaign`, `Step`, and/or `Workflow` to gather raw data from PanDa and the Butler, and store the data in members of the class. (In this step, what I have in mind is to capture the data returned from PanDa and the Bulter "as is", with no additional processing. Additional processing to produce what a human wants to see is in step 4, below. If the collection and processing are separated, it makes it easier to change them independently, and saving the unprocessed data makes sure everything that can be recorded, is.)
3. Expand the `from_*` and `to_*` methods to save and load files with the data gathered in 2, if present.
4. Add new methods (probably [property methods](https://docs.python.org/3/howto/descriptor.html#properties)) to process the data captured in 2 and create `pandas.DataFrames` with the data as it should be presented to a user.
5. Expand the `to_jira` methods of each to use `tabulate` to convert `pandas.DataFrames` generated by the step 4 methods to `jira` markup, and add them to the description or comments of the issues.
6. Make CLI commands (using click) built on these classes that do things like 
 - create campaigns, steps, and workflows from scratch
 
update campaigns, steps, and workflows in jira based on local directories, and vice versa
 6.2 add and remove steps from campaigns
 6.4 add and remove workflows from steps
 6.5 print tables generated by step 4 methods to the terminal